In [1]:
import numpy as np
import pandas as pd
import holidays
import optuna

from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold 
from sklearn.metrics import mean_absolute_percentage_error, make_scorer

In [2]:
df = pd.read_csv('data/train.csv')
df_pred = pd.read_csv('data/test.csv')

In [3]:
# Привожу дату к типу pd.datetime
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df_pred['date'] = pd.to_datetime(df_pred['date'], format='%Y-%m-%d')

In [4]:
# Новые признаки из даты
df['day'] = df.date.dt.day
df['month'] = df.date.dt.month
df['year'] = df.date.dt.year
# df['quarter'] = df.date.dt.quarter
# df['dayofyear'] = df.date.dt.dayofyear
# df['weekday'] = df.date.dt.weekday

df_pred['day'] = df_pred.date.dt.day
df_pred['month'] = df_pred.date.dt.month
df_pred['year'] = df_pred.date.dt.year
# df_pred['quarter'] = df_pred.date.dt.quarter
# df_pred['dayofyear'] = df_pred.date.dt.dayofyear
# df_pred['weekday'] = df_pred.date.dt.weekday

In [5]:
# holiday_dict = {
#     'Canada': holidays.CA(),
#     'Finland': holidays.FI(),
#     'Italy': holidays.IT(),
#     'Kenya': holidays.KE(),
#     'Norway': holidays.NO(),
#     'Singapore': holidays.SG()
# }


# def is_holiday(row):
#     country = row['country']
#     date = row['date']
    
#     if date in holiday_dict[country]:
#         return True
#     else:
#         return False


# def is_weekend(row):
#     date = row['date']

#     if date.weekday() >= 5:
#         return True
#     else:
#         return False


# df['weekend'] = df.apply(is_weekend, axis=1)
# df['holiday'] = df.apply(is_holiday, axis=1)

# df_pred['weekend'] = df_pred.apply(is_weekend, axis=1)
# df_pred['holiday'] = df_pred.apply(is_holiday, axis=1)

In [6]:
# Удаляю id
df.drop(columns='id', inplace=True)
df_pred.drop(columns='id', inplace=True)

# Удаляю Nan значения
df.dropna(subset=['num_sold'], inplace=True)

In [7]:
# удаляю date
df.drop(columns='date', inplace=True)
df_pred.drop(columns='date', inplace=True)

In [8]:
lst = ['country', 'store', 'product']
df = pd.get_dummies(df, columns=lst, dtype='bool')
df_pred = pd.get_dummies(df_pred, columns=lst, dtype='bool')

In [9]:
X = df.drop(columns='num_sold')
y = df['num_sold']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [10]:
# Синхронизация столбцов
df_pred = df_pred[X.columns]

In [11]:
# def objective(trial: optuna.Trial):
#     '''
#     Обычная сигнатура функции оптимизации для optuna.
#     '''
#     param = {
#             "metric": 'MAPE',
#             "verbosity": -1,
#             "random_state": 42,
#             "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
#             "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
#             'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
#             'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
#             "max_depth": trial.suggest_int("max_depth", 5, 25),
# #            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 0.9),
# #            "subsample": trial.suggest_float("subsample", 0.8, 1.0),
#             "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
#             'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
#             'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0)}
    
#     lgbm_model = LGBMRegressor(**param)
#     scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)
#     kf = KFold(n_splits=5, shuffle=True, random_state=42)
#     cv_scores = cross_val_score(lgbm_model, X_train, y_train, cv=kf, scoring=scorer)
#     return -np.mean(cv_scores)
    


# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=100)

In [12]:
best_params = {'learning_rate': 0.05633483952355363,
 'n_estimators': 706,
 'lambda_l1': 0.007111819841668459,
 'lambda_l2': 1.8352717695707565e-05,
 'max_depth': 13,
 'min_child_samples': 9,
 'feature_fraction': 0.6958643949900152,
 'bagging_fraction': 0.4212591389800655}

In [13]:
lgbmr_model = LGBMRegressor(**best_params)

lgbmr_model.fit(X_train, y_train, eval_set=(X_test, y_test))
y_pred = lgbmr_model.predict(X_test)
mae = mean_absolute_percentage_error(y_test, y_pred)
pred = lgbmr_model.predict(df_pred)
mae

[LightGBM] [Warning] feature_fraction is set=0.6958643949900152, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6958643949900152
[LightGBM] [Warning] lambda_l1 is set=0.007111819841668459, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.007111819841668459
[LightGBM] [Warning] lambda_l2 is set=1.8352717695707565e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.8352717695707565e-05
[LightGBM] [Warning] bagging_fraction is set=0.4212591389800655, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4212591389800655
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] feature_fraction is set=0.6958643949900152, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6958643949900152
[LightGBM] [Warning] lambda_l1 is set=0.007111819841668459, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.007111819841668459
[LightGBM] [Warning] lambda_l2 is set=1.835271769

0.14161237525932033

In [14]:
df_pred_duble = pd.read_csv('data/test.csv')
df_pred_Id = df_pred_duble['id']

output = pd.DataFrame({'id': df_pred_Id, 'num_sold': pred})
output.to_csv('data/lgbmr_model.csv', index=False)